# Offer data rating
### Input:

In [ ]:
host = 'localhost'
database='roomekbot$offers'
user='root'
password='Ad4Gw2'
city = 'Warszawa'
sql_query = f"select * from offers where city = '{city}'" # and business_type = 'buy'"

### Import database

In [ ]:
import mysql.connector
from mysql.connector import Error

try:
    connection = mysql.connector.connect(host=host, database=database, user=user, password=password)
    cursor = connection.cursor()
    cursor.execute(sql_query)
    records = cursor.fetchall()
    
    sql_query = "describe offers"
    cursor = connection.cursor()
    cursor.execute(sql_query)
    titles = cursor.fetchall()
    titles = [x[0] for x in titles]
    
except Error as e:
    print("Error reading data from MySQL table", e)
finally:
    if (connection.is_connected()):
        connection.close()
        cursor.close()

### Create dataframe

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.DataFrame(data=records, columns=titles)

df.head()

### Get center of the city

In [ ]:
import requests
import json


req = requests.get(url=f"https://nominatim.openstreetmap.org/search/pl/{city}?format=json&addressdetails=1&limit=1")

lat=float(json.loads(req.text)[0]['lat'])
lon=float(json.loads(req.text)[0]['lon'])

### Create the map

In [ ]:
import folium

m = folium.Map(
    location=[lat, lon],
    tiles='OpenStreetMap',   # OpenStreetMap, Stamen Terrain, Stamen Toner, Mapbox Bright, Mapbox Control Room
    zoom_start=12,
    control_scale=True
)

### Clusters for zooming out

In [ ]:
from folium.plugins import MarkerCluster # FastMarkerCluster

Cluster_x = MarkerCluster()
m.add_child(Cluster_x)

for x in range(len(df['location_latitude'])):
    Cluster_x.add_child(folium.Marker(location=[df['location_latitude'][x], df['location_longitude'][x]], popup=str(df['price'][x])))

In [ ]:
# for x in range(len(df['location_latitude'])):
#     folium.Marker(location=[df['location_latitude'][x], df['location_longitude'][x]], popup=str(df['price'][x])).add_to(m)

m

### Addditional objects

In [ ]:
##### Circle
# folium.Circle(
#     radius=100,
#     location=[52.24, 21.05],
#     popup='Test',
#     color='crimson',
#     fill=False,
# ).add_to(m)

In [ ]:
##### CircleMarker
# folium.CircleMarker(
#     location=[52.20, 21.01],
#     radius=50,
#     popup='Test',
#     color='#3186cc',
#     fill=True,
#     fill_color='#3186cc'
# ).add_to(m)

### Minimap

In [ ]:
##### MiniMap
# from folium.plugins import MiniMap
# minimap = MiniMap()
# m.add_child(minimap)

### Prices heatmap

In [ ]:
import numpy as np
from folium.plugins import HeatMap

datax = [df['location_latitude'], df['location_longitude'], df['price']/max(df['price'])]
datax = list(map(list, zip(*datax)))

In [ ]:
HeatMap(data=datax, name="Ceny", min_opacity=0.5, max_zoom=16, max_val=1.0, radius=100, blur=20).add_to(m)

m

# m.save(os.path.join('results', 'Heatmap.html'))

### Save to file

In [ ]:
m.save('index.html')

### Show average price isolines

In [ ]:
# TODO show average price isolines

### Colorful districts

In [ ]:
# def my_color_function(feature):
#     """Maps low values to green and high values to red."""
#     if unemployment_dict[feature['id']] > 6.5:
#         return '#ff0000'
#     else:
#         return '#008000'
# m = folium.Map([43, -100], tiles='cartodbpositron', zoom_start=4)

# folium.GeoJson(
#     geo_json_data,
#     style_function=lambda feature: {
#         'fillColor': my_color_function(feature),
#         'color': 'black',
#         'weight': 2,
#         'dashArray': '5, 5'
#     }
# ).add_to(m)

# m.save(os.path.join('results', 'Colormaps_0.html'))

# m